# Visual Unit Tests for BFGS Optimizer


In [8]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from optim.algorithms.bfgs import BFGS

NUM_ITER = 100

def create_contours(model, x1, x2):
    X1, X2 = np.meshgrid(x1, x2)
    Z = np.zeros_like(X1)
    for i in range(len(x1)):
        for j in range(len(x2)):
            x = torch.tensor([x1[i], x2[j]], dtype=torch.float32)
            Z[j,i] = model(x)
    return X1, X2, Z

def plot_traj(traj, title, contours=None):
    plt.figure()
    # plot contours
    if contours is not None:
        X1, X2, Z = contours
        plt.contour(X1, X2, Z, levels=20)
        plt.colorbar(label='f(x)')
    # plot trajectory
    plt.plot(traj[:,0], traj[:,1])
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(title)
    plt.grid(True)
    plt.show()

def optimize(model, x, optimizer, loss_cl=None):
    traj = []
    traj.append(x.detach().numpy().copy())

    # optimize
    for _ in range(NUM_ITER):
        optimizer.zero_grad()
        loss = model(x)
        loss.backward()
        optimizer.step(loss_cl=loss_cl)
        traj.append(x.detach().numpy().copy())

    return np.array(traj)

## Simple Quadratic

$f(x) = x^T x$

In [9]:
from optim.functions.quadratic import Quadratic

# create model
A = 2 * torch.eye(2, dtype=torch.float32)
b = torch.zeros(2, dtype=torch.float32)
c = torch.tensor(0.0, dtype=torch.float32)
model = Quadratic(A, b, c)

# create contours
x1 = np.linspace(-1.5, 1.5, 25)
x2 = np.linspace(-1.5, 1.5, 25)
contours = create_contours(model, x1, x2)

#### Constant Step Size 

In [ ]:
# create input
x = torch.tensor([1.0, 1.0], dtype=torch.float32, requires_grad=True)

# optimize 
traj = optimize(model, x, BFGS([x], step_type='constant', step_size=1e-1))

plot_traj(traj, "BFGS | Simple Quadratic | Constant", contours)

#### Backtracking 

In [ ]:
# create input
x = torch.tensor([1.0, 1.0], dtype=torch.float32, requires_grad=True)

# optimize 
traj = optimize(model, x, BFGS([x], step_type='armijo', alpha=1), loss_cl=lambda: model(x))

plot_traj(traj, "BFGS | Simple Quadratic | Armijo", contours)

## Rosenbrock

$f(x) = (1-x_1)^2 + 100(x_2-x_1^2)^2$

In [12]:
from optim.functions.rosenbrock import Rosenbrock

# create model
model = Rosenbrock()

# create contours
x1 = np.linspace(-2, 2, 25)
x2 = np.linspace(-3, 3, 25)
contours = create_contours(model, x1, x2)

#### Constant Step Size 

In [ ]:
# create input
x = torch.tensor([-1.2, 1.0], dtype=torch.float32, requires_grad=True)

# optimize 
traj = optimize(model, x, BFGS([x], step_type='constant', step_size=1e-3))

plot_traj(traj, "BFGS | Rosenbrock | Constant", contours)

#### Backtracking

In [ ]:
# create input
x = torch.tensor([-1.2, 1.0], dtype=torch.float32, requires_grad=True)

# optimize 
traj = optimize(model, x, BFGS([x], step_type='armijo', alpha=1), loss_cl=lambda: model(x))

plot_traj(traj, "BFGS | Rosenbrock | Armijo", contours)